# Testing the waters

Can ignore this section of code

In [2]:
from keras.models import Sequential
from keras.layers import Dense
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import glob
import cv2
import numpy as np
import PIL


/Users/leannwoo/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=100))
model.add(Dense(units=10, activation='softmax'))

In [4]:
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [5]:
datagen = ImageDataGenerator(rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            rescale=1./255,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')

In [6]:
img = load_img('/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Eyespots_all_species/eyespot/Apogon_Apogon cavitiensis_-1692469642.jpg')

In [7]:
x = img_to_array(img)
x = x.reshape((1,) + x.shape)

In [12]:
i = 0
for batch in datagen.flow(x, batch_size=1,
                         save_to_dir='preview', save_prefix='eyespot', save_format='jpeg'):
    i += 1
    if i>20:
        break

# Building our model

Start paying attention here

Solid explanation of kfold cross validation: https://www.openml.org/a/estimation-procedures/1  
Code based on: https://www.kaggle.com/stefanie04736/simple-keras-model-with-k-fold-cross-validation

In [2]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.utils import to_categorical
from keras import backend as K
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
import glob
import cv2

/Users/leannwoo/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [35]:
images = glob.glob('*.jpg')
len(images)


def getFish(filepath):
    img = cv2.imread(filepath)
    # standardize size for prediction step
    img = cv2.resize(img, (150, 150))
    img = img/255.0
    img = img.reshape((1,) + img.shape)
    
    classification, _, species = filepath.partition('_')
    
    #res = model.predict(img)
    return(img, classification, species)

def load_data_kfold(k, image_filepaths):
    
    #use getFish function 
    #load in one file, append to a list
    
    x_train = []
    y_train = []
    
    for i in range(len(image_filepaths)):
        x, y, _ = getFish(image_filepaths[i])
        x_train.append(x)
        y_train.append(y)
        if (y_train[i] == 'eyespot'):
            y_train[i] = 0
        else:
            y_train[i] = 1
    
#     train = pd.read_json('../input/train.json')
#     train.inc_angle = train.inc_angle.replace('na', 0)
    
#     x_band1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
#     x_band2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
#     x_band3 = x_band1 / x_band2
       
#     X_train = np.concatenate([x_band1[:, :, :, np.newaxis]
#                             , x_band2[:, :, :, np.newaxis]
#                             , x_band3[:, :, :, np.newaxis]], axis=-1)
                         
#     y_train = np.array(train["is_iceberg"])
    
    #StratifiedKFold can only work in binary; convert labels to categorical afterwards
    folds = list(StratifiedKFold(n_splits=k, shuffle=True, random_state=1).split(x_train, y_train))
    
    y_train = to_categorical(y_train)
    
    return folds, x_train, y_train

k = 10
folds, x_train, y_train = load_data_kfold(k, images)

In [68]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Eyespots_all_species/'
validation_data_dir = '/Users/leannwoo/Dropbox/machine_learning_fishes/image_classifiers/Eyespots_all_species_validation/'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 2
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('sigmoid'))

model.compile(loss='mean_squared_error',
              optimizer='rmsprop',
              metrics=['accuracy'])



# DO NOT RUN THIS BLOCK IF USING KFOLD CV

In [81]:
# # this is the augmentation configuration we will use for training
# train_datagen = ImageDataGenerator(
#     rescale=1. / 255,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True)

# # this is the augmentation configuration we will use for testing:
# # only rescaling
# test_datagen = ImageDataGenerator(rescale=1. / 255)

# train_generator = train_datagen.flow_from_directory(
#     train_data_dir,
#     target_size=(img_width, img_height),
#     batch_size=batch_size,
#     class_mode='categorical')

# validation_generator = test_datagen.flow_from_directory(
#     validation_data_dir,
#     target_size=(img_width, img_height),
#     batch_size=batch_size,
#     class_mode='categorical')

# train_generator[1][0][0].shape

In [79]:
gen = ImageDataGenerator(horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.1,
                         height_shift_range = 0.1,
                         zoom_range = 0.1,
                         rotation_range = 10
                        )

for j, (train_idx, val_idx) in enumerate(folds):
    
    X_train_cv = []
    y_train_cv = []
    X_valid_cv = []
    y_valid_cv = []
    
    for i in range(len(train_idx)):
        X_train_cv.append(x_train[train_idx[i]])
        y_train_cv.append(y_train[train_idx[i]])
    for i in range(len(val_idx)):
        X_valid_cv.append(x_train[val_idx[i]])
        y_valid_cv.append(y_train[val_idx[i]])
    
    X_train_cv = np.array(X_train_cv)
    X_train_cv = np.squeeze(X_train_cv, axis = 1)
    
    X_valid_cv = np.array(X_valid_cv)
    X_valid_cv = np.squeeze(X_valid_cv, axis = 1)
    
    y_train_cv = np.array(y_train_cv)
    y_valid_cv = np.array(y_valid_cv)
    
    print '\nFold ',j
    name_weights = "final_model_fold" + str(j) + "_weights.h5"
    generator = gen.flow(X_train_cv, y_train_cv, batch_size = batch_size)
    model.fit_generator(
                generator,
                steps_per_epoch=len(X_train_cv)/batch_size,
                epochs=2,
                shuffle=True,
                verbose=1,
                validation_data = (X_valid_cv, y_valid_cv))
    model.save_weights('first_try.h5')
    model.save('second_try.h5')
    print(model.evaluate(X_valid_cv, y_valid_cv))


Fold  0
Epoch 1/2
4/4 [==============================] - 3s 849ms/step - loss: 0.2482 - acc: 0.5286 - val_loss: 0.2410 - val_acc: 0.5000
Epoch 2/2
8/8 [==============================] - 0s 19ms/step
[0.26872968673706055, 0.5]

Fold  1
Epoch 1/2
4/4 [==============================] - 3s 716ms/step - loss: 0.2570 - acc: 0.5964 - val_loss: 0.1955 - val_acc: 0.8750
Epoch 2/2
8/8 [==============================] - 0s 14ms/step
[0.20348475873470306, 0.5]

Fold  2
Epoch 1/2
4/4 [==============================] - 3s 705ms/step - loss: 0.2445 - acc: 0.5482 - val_loss: 0.2268 - val_acc: 0.5000
Epoch 2/2
8/8 [==============================] - 0s 15ms/step
[0.22943437099456787, 0.625]

Fold  3
Epoch 1/2
4/4 [==============================] - 3s 690ms/step - loss: 0.2308 - acc: 0.5804 - val_loss: 0.2579 - val_acc: 0.3750
Epoch 2/2
8/8 [==============================] - 0s 14ms/step
[0.242535799741745, 0.625]

Fold  4
Epoch 1/2
4/4 [==============================] - 3s 663ms/step - loss: 0.2540 - a

# DO NOT RUN THIS BLOCK IF USING KFOLD CV

In [80]:
# model.fit_generator(
#     train_generator,
#     steps_per_epoch=nb_train_samples // batch_size,
#     epochs=epochs,
#     validation_data=validation_generator,
#     validation_steps=nb_validation_samples // batch_size)

# model.save_weights('first_try.h5')
# model.save('second_try.h5')

## Reload model and use it to predict

In [4]:
import cv2

def getFish(filepath):
    img = cv2.imread(filepath)
    # standardize size for prediction step
    img = cv2.resize(img, (150, 150))
    img = img/255.0
    img = img.reshape((1,) + img.shape)
    
    classification, _, species = filepath.partition('_')
    
    #res = model.predict(img)
    return(img, classification, species)

#run through all images, classify,
#and write out the classification category, 
#the confidence/weight of the prediction, 
#the species name, and the family. 

def predict(model, img):
    prediction_weights = model.predict(img[0])
    prediction = model.predict_classes(img[0])
    if (prediction[0] == 0):
        prediction = "eyespot"
    else:
        prediction = "noeyespot"
    fam = img[2]
    correct_class = img[1]
    #print "Model classification:",prediction,"\nCorrect classification:", correct_class,"\nPrediction wieghts:",prediction_weights,"\nType:",fam,"\n"
    return(prediction, correct_class, prediction_weights, fam)


In [5]:
from keras.models import load_model

mod = load_model("second_try.h5")

eyepath = "eyespot_Abudefduf_Abudefduf sparoides_929157376.jpg"
noeyepath = "noeyespot_Abudefduf_Abudefduf bengalensis_880932306.jpg"

predict(mod, getFish(eyepath))

('eyespot',
 'eyespot',
 array([[1.000000e+00, 5.364594e-38]], dtype=float32),
 'Abudefduf_Abudefduf sparoides_929157376.jpg')

In [6]:
import glob

images = glob.glob('*.jpg')
len(images)

80

In [7]:
import csv

with open('classification_data.csv', 'wb') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['Model Classification', 'Correct Classification', 'Prediction Weights', 'Type'])

    true_eye=0
    true_noeye=0
    false_eye=0
    false_noeye=0

    for i in range(len(images)):
        info = predict(mod, getFish(images[i]))
        filewriter.writerow([info[0], info[1], info[2], info[3]])

        #computing info for confusion matrix
        if (info[0] == info[1]):
            if (info[0] == 'eyespot'):
                true_eye += 1
            else:
                true_noeye += 1
        else:
            if (info[0] == 'eyespot'):
                false_eye += 1
            else:
                false_noeye += 1

In [8]:
import pandas as pd

data = {'Eyespots':[true_eye, false_noeye], 'No Eyespots':[false_eye, true_noeye] }
confusion_matrix = pd.DataFrame(data = data)
confusion_matrix = confusion_matrix.rename({0:'Eyespots (Model Output)', 1:'No Eyespots (Model Output)'},axis='index')
confusion_matrix

,Eyespots,No Eyespots
Eyespots (Model Output),30,4
No Eyespots (Model Output),10,36
